In [1]:
import cv2
import numpy as np
import os

In [2]:
path = 'data/sample/'

In [3]:
train_xml_files = [os.path.join(path + 'train_xml/', i) for i in os.listdir(path + 'train_xml/') 
                   if os.path.isfile(os.path.join(path + 'train_xml/', i))]

In [4]:
valid_xml_files = [os.path.join(path + 'valid_xml/', i) for i in os.listdir(path + 'valid_xml/') 
                   if os.path.isfile(os.path.join(path + 'valid_xml/', i))]

In [5]:
from xml.dom import minidom

In [29]:
xmldoc = minidom.parse(train_xml_files[0])

In [30]:
annotation = xmldoc.getElementsByTagName('annotation')[0]

In [31]:
filename = annotation.getElementsByTagName('filename')[0]
print(filename.firstChild.data)

2002_09_02_img_15510.jpg


In [32]:
file_sizes = annotation.getElementsByTagName('size')[0]
width = file_sizes.getElementsByTagName('width')[0]
height = file_sizes.getElementsByTagName('height')[0]

In [33]:
print(width.firstChild.data, height.firstChild.data)

293 450


In [34]:
obj = annotation.getElementsByTagName('object')[0]
bndbox = obj.getElementsByTagName('bndbox')[0]

xmin = bndbox.getElementsByTagName('xmin')[0]
xmin = int(xmin.firstChild.data)

ymin = bndbox.getElementsByTagName('ymin')[0]
ymin = int(ymin.firstChild.data)

xmax = bndbox.getElementsByTagName('xmax')[0]
xmax = int(xmax.firstChild.data)

ymax = bndbox.getElementsByTagName('ymax')[0]
ymax = int(ymax.firstChild.data)

In [35]:
print(xmin, ymin, xmax, ymax)

71 52 218 230


In [38]:
img = cv2.imread(path + 'train/face/' + filename.firstChild.data, cv2.COLOR_BGR2RGB)

# coordinates of the vertex at the top left (x, y)
# coordinates of the lower right vertex (x, y)
# 123.583300 85.549500 1.265839 269.693400 161.781200
# img = cv2.rectangle(img, (161, 20), (414, 291), (0,0,255), 2)
# img = cv2.rectangle(img, (161, 20), (414, 291), (0,255,0), 2)

img = cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (0,0,255), 2)

cv2.imshow('Image', img)
cv2.waitKey(0)

255

In [37]:
orig_width = int(width.firstChild.data)
orig_height = int(height.firstChild.data)

conv_width = 224. / orig_width
conv_height = 224. / orig_height

print(conv_width, conv_height)

new_xmin = int(xmin * conv_width)
new_ymin = int(ymin * conv_height)
new_xmax = int(xmax * conv_width)
new_ymax = int(ymax * conv_height)

print(new_xmin, new_ymin, new_xmax, new_ymax)

0.764505119453925 0.49777777777777776
54 25 166 114


In [39]:
img2 = cv2.imread(path + 'train/face/' + filename.firstChild.data, cv2.COLOR_BGR2RGB)

img2 = cv2.resize(img2, (224, 224))

img2 = cv2.rectangle(img2, (new_xmin, new_ymin), (new_xmax, new_ymax), (0,0,255), 2)

cv2.imshow('Image', img2)
cv2.waitKey(0)

255

# Collect all originals image info like filename, width, height, bound box

In [61]:
orig_bb_box = []

In [62]:
for file in train_xml_files:
    xmldoc = minidom.parse(file)
    
    annotation = xmldoc.getElementsByTagName('annotation')[0]
    
    filename = annotation.getElementsByTagName('filename')[0]
    filename = filename.firstChild.data
    
    file_sizes = annotation.getElementsByTagName('size')[0]
    width = file_sizes.getElementsByTagName('width')[0]
    width = int(width.firstChild.data)
    height = file_sizes.getElementsByTagName('height')[0]
    height = int(height.firstChild.data)
    
    obj = annotation.getElementsByTagName('object')[0]
    bndbox = obj.getElementsByTagName('bndbox')[0]

    xmin = bndbox.getElementsByTagName('xmin')[0]
    xmin = int(xmin.firstChild.data)

    ymin = bndbox.getElementsByTagName('ymin')[0]
    ymin = int(ymin.firstChild.data)

    xmax = bndbox.getElementsByTagName('xmax')[0]
    xmax = int(xmax.firstChild.data)

    ymax = bndbox.getElementsByTagName('ymax')[0]
    ymax = int(ymax.firstChild.data)
    
    orig_bb_box.append([width, height, xmin, ymin, xmax, ymax])

In [63]:
orig_bb_box = np.array(orig_bb_box)

In [64]:
orig_bb_box.shape, orig_bb_box[0:1]

((261, 6), array([[293, 450,  71,  52, 218, 230]]))

In [68]:
resized_bb_box = []

In [69]:
for i in orig_bb_box:
    conv_width = 224. / i[0]
    conv_height = 224. / i[1]

    new_xmin = int(xmin * conv_width)
    new_ymin = int(ymin * conv_height)
    new_xmax = int(xmax * conv_width)
    new_ymax = int(ymax * conv_height)

    resized_bb_box.append([new_xmin, new_ymin, new_xmax, new_ymax])

In [70]:
resized_bb_box = np.array(resized_bb_box)

In [71]:
resized_bb_box.shape, resized_bb_box[:1]

((261, 4), array([[ 51,   3, 128,  77]]))